In [ ]:
from bs4 import BeautifulSoup as bs
from lxml.html import fromstring
from requests import get
from urllib.parse import urlencode, urlparse, parse_qs
from rake_nltk import Rake
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import argparse
import datetime
import requests
import json
import urllib
import spacy
import pytextrank
import re
import time
import os
import timeit

In [ ]:
def preprocess_text(sen):

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
event_data = pd.read_csv("CWAL.csv", nrows=423)

# event_data = pd.read_csv("covid19/covid_data_with_tag.csv")

# event_data = event_data[event_data['sentence'].notna()]

# event_data = event_data.loc[(event_data.importance!=0) & (event_data.tag!='Other')]
# event_data.shape

# event_data.head()
event_data.cluster.nunique()

In [ ]:
import spacy
from spacy import displacy
from spacy.lemmatizer import Lemmatizer
from collections import Counter
import pandas as pd
import numpy as np
import time

import en_core_web_sm
nlp = en_core_web_sm.load()

from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB

lemmatizer = nlp.vocab.morphology.lemmatizer


actions = []

for sent in event_data.sentence.to_list():
    doc = nlp(sent)

    hw = [x.lemma_ for x in [y 
                            for y
                            in doc
                            if not y.is_stop and y.pos_ != 'PUNCT'] if x.pos_ == 'VERB' and x.dep_ == 'ROOT']

    hw = list(set(hw))

    actions.append(hw)
    
for i in range(len(actions)):
    actions[i] = str(actions[i])[1:-1]
    
event_data['action'] = pd.Series(actions)

event_data['index'] = event_data.index
event_data.head()


In [ ]:
# event_file = file.loc[(file.tag=='event') | (file.tag=='fact') ]
event_data = event_data[event_data['sentence'].notna()]
event_data = event_data[event_data['cluster'].notna()]
event_data.head(), event_data.shape

In [ ]:
from bs4 import BeautifulSoup
!pip install google
!pip install rake_nltk --user
!pip install pytextrank
from googlesearch import search
import requests


def google_scrape(url):
    thepage = requests.get(str(url)).text
    soup = BeautifulSoup(thepage, "html.parser")
    try:
        txt = soup.title.text
    except:
        txt = ""
    return txt

In [ ]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# nltk.download('stopwords')
# nltk.download('all')
from nltk.corpus import stopwords
#get english stopwords
en_stopwords = set(stopwords.words('english'))

# Adding custom stopwords
en_stopwords.update({'LETTER', 'COLLECTED', 'WORKS', 'MAHATMA',  'GANDHI', '|'})

#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

tag_list = []


for i in event_data.index:
    
    print("%s processing"%i)
    
    query = str(event_data.sentence[i]).strip()
#     time.sleep(5)

    start_time = timeit.default_timer()
    my_results_list = []
    for url in search(query + 'gandhi',        # The query you want to run
                    tld = 'com',  # The top level domain
                    lang = 'en',  # The language
                    num = 5,     # Number of results per page
                    start = 0,    # First result to retrieve
                    stop = 5,
                    safe='on',  # Last result to retrieve
                    pause = 5.0,  # Lapse between HTTP requests
                ):
        try:
            r = google_scrape(url)
        except:
            r = ''
        my_results_list.append(r)




    text = query + " " + " ".join(my_results_list)
    
    file = str(event_data['book no.'][i]).strip()
    with open(str(os.path.join('text_files_combined', file)), 'r', encoding="utf8") as content:
        raw_data = content.read()

    raw_data = str(event_data.Text[i])

    text =  raw_data + " ".join(my_results_list) 
    print(text)

    text = text.replace('LETTER TO', '')
    text = text.replace('COLLECTED WORKS', ' ')
    text = text.replace('MAHATMA GANDHI', '')

    elapsed = timeit.default_timer() - start_time
    print(elapsed)

    tokens = nltk.tokenize.word_tokenize(text)
    bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(tokens)

    #bigrams
    bigram_freq = bigramFinder.ngram_fd.items()
    bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

    #filter bigrams
    filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]
    tags = filtered_bi['bigram'].tolist()[:15]
    tags = [" ".join(x) for x in tags]
    print(query)
    print(tags)

    tag_list.append(tags)



In [ ]:
# removing '[' from tag
  
for i in range(len(tag_list)):
    tag_list[i] = str(tag_list[i])[1:-1]

In [ ]:
# event_data.sentence[event_data.index[0]]

# tag_list1 = tag_list

# len(tag_list)
tag_list[0]

In [ ]:
# event_data.keywords = event_data.keywords.fillna('null')
# for i in event_data.index:
#     if event_data.keywords[i] == 'null':
#         event_data.keywords[i] = tag_list[i]
# event_data = event_data.loc[(event_data.importance!=0) & (event_data.tag!='Other')]
event_data['keywords_unfiltered'] = pd.Series(tag_list)

# event_data['index'] = event_data.index
event_data.head()
event_data.tag.value_counts()

In [ ]:
nlp = spacy.load("en_core_web_sm")
tr = pytextrank.TextRank(logger=None)
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

tags = []
for query in event_file.sentence[50:52]:
    # search_results = google.search(query, 2)
    # time.sleep(5)
    # text = query + " "
    # for res in search_results:
    #     text += res.name + " "
    #     text += res.description + " "

    my_results_list = []
    for url in search(query + 'gandhi',        # The query you want to run
                    tld = 'com',  # The top level domain
                    lang = 'en',  # The language
                    num = 5,     # Number of results per page
                    start = 0,    # First result to retrieve
                    stop = 5,
                    safe='on',  # Last result to retrieve
                    pause = 5.0,  # Lapse between HTTP requests
                ):
        try:
            r = google_scrape(url)
        except:
            r = ''
        my_results_list.append(r)
        
    text = query + " " + " ".join(my_results_list)
    
    nlp.max_length = 10**7
    preprocessed_text = preprocess_text(text)
    doc = nlp(preprocessed_text)
    tag_list = []
    # for p in doc._.phrases[:10]:
    #     tag_list.append(p.text)
    # tags.append(tag_list)
    rake = Rake(max_length=3)
    rake.extract_keywords_from_text(preprocessed_text)
    tags.append(rake.get_ranked_phrases_with_scores()[:10])
    

In [ ]:
print(tags)

In [ ]:
# my_tag_list = []
my_tag_list += tags
my_tag_list

In [ ]:
tag_list = [[]]* event_file.shape[0]
for i in range(len(my_tag_list)):
    tag_list[i] = my_tag_list[i]
event_file['knowledge_tag'] = tag_list

# event_file.shape
event_file.head()

In [ ]:
event_file.to_csv('sentence_with_knowledge_tag.csv')

In [ ]:
# from rake_nltk import Rake
tr = pytextrank.TextRank(logger=None)
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)
nlp.max_length = 10**7
preprocessed_text = preprocess_text(text)
doc = nlp(preprocessed_text)

for p in doc._.phrases[:10]:
    print("{}".format(p.text))
    
rake = Rake(max_length=2)
rake.extract_keywords_from_text(preprocessed_text)
print(rake.get_ranked_phrases_with_scores()[:10])

In [ ]:
selected = event_file.loc[(file.tag=='event') | (file.tag=='fact') | (file.tag=='demand')]
selected.head()

In [ ]:
# q = str(event_file.sentence[2]).strip()

# time.sleep(5)
# search_results = google.search(q, 3)

# text = query + " "
# for res in search_results:
#     print(1)
#     text += res.name + " "
#     text += res.description + " "
    
# text += event_file.Text[2]

text = event_file.Text[2]

In [ ]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# nltk.download('stopwords')
from nltk.corpus import stopwords
#get english stopwords
en_stopwords = set(stopwords.words('english'))
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in en_stopwords or word.isspace():
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False
    
tag_list = []

for i in event_file.id: 
    if event_file.tag[i] == 'event' or event_file.tag[i] == 'fact' or event_file.tag[i] == 'demand':
        text = preprocess_text(event_file.Text[i])
        tokens = nltk.tokenize.word_tokenize(text + event_file.sentence[i])
        bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(tokens)
    #     trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(tokens)

        # change this to read in your data
        # finder = BigramCollocationFinder.from_words(file.Text[0])
        # # only bigrams that appear 3+ times
        # finder.apply_freq_filter(3) 

        # return the 5 n-grams with the highest PMI
        # finder.nbest(bigram_measures.pmi, 5)  

        #bigrams
        bigram_freq = bigramFinder.ngram_fd.items()
        bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)

        #filter bigrams
        filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]
        tags = filtered_bi['bigram'].tolist()
        tags = [" ".join(x) for x in tags]

        tag_list.append(tags[:10])
    else:
        tag_list.append(np.nan)

# #trigrams
# trigram_freq = trigramFinder.ngram_fd.items()
# trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)

# #function to filter for trigrams
# def rightTypesTri(ngram):
#     if '-pron-' in ngram or 't' in ngram:
#         return False
#     for word in ngram:
#         if word in en_stopwords or word.isspace():
#             return False
#     first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
#     third_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
#     tags = nltk.pos_tag(ngram)
#     if tags[0][1] in first_type and tags[2][1] in third_type:
#         return True
#     else:
#         return False
# #filter trigrams
# filtered_tri = trigramFreqTable[trigramFreqTable.trigram.map(lambda x: rightTypesTri(x))]

In [ ]:
tag_list

In [ ]:
for i in range(len(tag_list)):
    tag_list[i] = str(tag_list[i])[1:-1]

event_file['keywords'] = tag_list



In [ ]:
from nltk import bigrams
from nltk import trigrams
# #filter for only those with more than 20 occurences
# bigramFinder.apply_freq_filter(20)
# trigramFinder.apply_freq_filter(20)
bigramPMITable = pd.DataFrame(list(bigramFinder.score_ngrams(bigram_measures.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)
# trigramPMITable = pd.DataFrame(list(trigramFinder.score_ngrams(bigram_measures.pmi)), columns=['trigram','PMI']).\
#                                     sort_values(by='PMI', ascending=False)

In [ ]:
bigramPMITable